In [ ]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [ ]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [ ]:
leagues_model.load_counties_data()

### Blank Sandbox

In [ ]:
leagues_model.reset_county_styles()

(leaflet_map, geojson_layer) = leagues_model.render_map(None)

leaflet_map

In [ ]:
from sportwarz import Team, Coordinates, LeagueJson

import json
import codecs
import ipywidgets as widgets

leagues = { }

league_dropdown_options = []

league_input = widgets.Dropdown(options = league_dropdown_options)
name_input = widgets.Text()
venue_input = widgets.Text()
L_input = widgets.FloatText()
S_input = widgets.FloatText()
N_input = widgets.FloatText()
lat_input = widgets.FloatText()
lon_input = widgets.FloatText()
color_input = widgets.ColorPicker()
state_input = widgets.Text()
add_button = widgets.Button(description="Add Team")
error_message = widgets.Label(style = { "color" : "red" })

out = widgets.Output()
out.clear_output()

def add_new_team(button):
    button.disabled=True
    try:
        if not league_input.value in leagues_model._leagues:
            error_message.value = "League not specified"
            return
        if not name_input.value.strip():
            error_message.value = "Name not specified"
            return
        if not L_input.value:
            error_message.value = "L not specified"
            return
        if not S_input.value:
            error_message.value = "S not specified"
            return
        if not N_input.value:
            error_message.value = "N not specified"
            return
        if not venue_input.value:
            error_message.value = "Venue not specified"
            return
        if not lon_input.value:
            error_message.value = "Lon not specified"
            return
        if not lat_input.value:
           error_message.value = "Lat not specified"
           return
    
        newTeam = Team(
            name = name_input.value.strip(),
            L = L_input.value,
            S = S_input.value,
            N = N_input.value,
            venue = venue_input.value,
            state = state_input.value,
            color = color_input.value,
            coordinates = Coordinates(
                lat = lat_input.value,
                lon = lon_input.value
        ))
        leagues_model.add_team(league_input.value, [newTeam])
        leagues_model.calculate_distances() 
        leagues_model.compute_shares()  
        leagues_model.compute_output_dataframe() 
    
        leagues_model.heatmap_counties(league_input.value)
        new_layer = leagues_model.replace_geojson_layer(leaflet_map, geojson_layer)  
        geojson_layer = new_layer

    finally:
        button.disabled=False

add_button.on_click(add_new_team)

add_panel = widgets.VBox([widgets.GridBox([
    widgets.Label(value='League:'), league_input,
    widgets.Label(value='Name:'), name_input,
    widgets.Label(value='Venue:'), venue_input,
    widgets.Label(value='L:'), L_input,
    widgets.Label(value='S:'), S_input,
    widgets.Label(value='N:'), N_input,
    widgets.Label(value='Lat:'), lat_input,
    widgets.Label(value='Lon:'), lon_input,
    widgets.Label(value='Color:'), color_input,
    widgets.Label(value='State:'), state_input,
    add_button
],layout=widgets.Layout(grid_template_columns="repeat(2, 100px)")),
    error_message
])

upload = widgets.FileUpload(
    accept='.json', 
    multiple= False  
)

delete_league_input = widgets.Dropdown()
delete_team_input = widgets.Dropdown()

delete_button = widgets.Button(description="Delete Team")

delete_panel = widgets.GridBox([
    widgets.Label(value='League:'), delete_league_input,  
    widgets.Label(value='Team:'), delete_team_input, 
    delete_button 
], layout=widgets.Layout(grid_template_columns="repeat(2, 100px)"))

@out.capture()
def delete_team(button):
    button.disabled = True
    try:
        if leagues_model.delete_teams(delete_league_input.value, [delete_team_input.value]):
            leagues_model.calculate_distances() 
            leagues_model.compute_shares()  
            leagues_model.compute_output_dataframe() 
    
            leagues_model.heatmap_counties(delete_league_input.value)
            new_layer = leagues_model.replace_geojson_layer(leaflet_map, geojson_layer)  
            geojson_layer = new_layer

    finally:
        button.disabled = False

@out.capture()
def delete_league_change(change):
    delete_team_input.options = list(map(lambda team: team["name"], leagues_model._leagues[delete_league_input.value]["teams"]))

delete_league_input.observe(delete_league_change, names='value')    

delete_button.on_click(delete_team)

@out.capture()
def on_upload_change(change):
    upload.disabled = True
    try:
        file_info = change["new"][0]
        content = codecs.decode(file_info.content, encoding="utf-8")
        league_json: LeagueJson = json.loads(content)
        league_name = league_json["league_name"]
        leagues_model.add_league(league_json)
        leagues_model.calculate_distances() 
        leagues_model.compute_shares()  
        leagues_model.compute_output_dataframe() 
    
        leagues_model.heatmap_counties(league_name)
        new_layer = leagues_model.replace_geojson_layer(leaflet_map, geojson_layer)  
        geojson_layer = new_layer         
        if league_name not in league_dropdown_options:
            league_dropdown_options.append(league_name)    
        league_input.options = league_dropdown_options
        delete_league_input.options = league_dropdown_options
    finally:
        upload.disabled=False   

upload.observe(on_upload_change, names="value")

upload_panel = widgets.GridBox([
    widgets.Label(value='File Upload:'), upload,    
], layout=widgets.Layout(grid_template_columns="repeat(2, 100px)"))

tab = widgets.Tab()
tab.children = [upload_panel, add_panel, delete_panel]
tab.titles =   ["Upload", "Add", "Delete"]
tab


In [ ]:
out